In [18]:
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph,START,END
from dotenv import load_dotenv
from typing import TypedDict
from langchain_core.output_parsers import StrOutputParser
load_dotenv()

True

In [29]:
model=ChatGroq(model="llama-3.1-8b-instant")
parser=StrOutputParser()

In [30]:
class BlogState(TypedDict):
    topic:str
    outline:str
    content:str

In [31]:
def create_outline(state:BlogState):
    topic=state["topic"]
    prompt=f"generate a outline for a blog around the topic:{topic}"
    state["outline"]=parser.invoke(model.invoke(prompt))
    return state

In [32]:
def generate_blog(state:BlogState):
    outline=state["outline"]
    topic=state["topic"]
    prompt=f"generate a blog around the topic:{topic} while strictly following this outline:{outline}"
    state["content"]=parser.invoke(model.invoke(prompt))
    return state

In [33]:
graph=StateGraph(BlogState)

graph.add_node("create_outline",create_outline)
graph.add_node("generate_blog",generate_blog)

graph.add_edge(START,"create_outline")
graph.add_edge("create_outline","generate_blog")
graph.add_edge("generate_blog",END)

workflow=graph.compile()

In [41]:
inital_state={"topic":"harry potter"}
workflow.invoke(inital_state)

{'topic': 'harry potter',
 'outline': 'Here\'s a suggested outline for a blog about Harry Potter:\n\n**Title:** "The Wizarding World of Harry Potter: Exploring Magic, Friendship, and Adventure"\n\n**I. Introduction**\n\n* Brief overview of the Harry Potter series\n* Explanation of why the series has become a cultural phenomenon\n* Thesis statement: Harry Potter is more than just a series of books - it\'s a world of magic, friendship, and adventure that continues to captivate audiences of all ages.\n\n**II. The Magic of the Wizarding World**\n\n* Overview of the magical world created by J.K. Rowling\n* Discussion of the different magical creatures, spells, and potions featured in the series\n* Analysis of how the magical world reflects real-world issues, such as prejudice and power struggles\n\n**III. The Power of Friendship**\n\n* Analysis of the relationships between Harry, Ron, and Hermione\n* Discussion of the themes of loyalty, trust, and sacrifice in the series\n* Exploration of h